In [1]:
!git clone https://github.com/MishaelTech/series_analyzing_system.git

Cloning into 'series_analyzing_system'...
remote: Enumerating objects: 314, done.
remote: Counting objects: 100% (314/314), done.
remote: Compressing objects: 100% (282/282), done.
remote: Total 314 (delta 39), reused 298 (delta 23), pack-reused 0 (from 0)
Receiving objects: 100% (314/314), 2.97 MiB | 10.17 MiB/s, done.
Resolving deltas: 100% (39/39), done.


In [3]:
!cd series_analyzing_system && pip install -r requirements.txt

In [ ]:
!pip install pyngrok streamlit

In [ ]:
from pyngrok import ngrok

# Start a ngrok tunnel to the streamlit app
ngrok.set_auth_token("2n43ceCJmLANUY539SUTSFa9vFX_4q5A4gskCusEmbkDPAWkp")  # Set your ngrok auth token here if needed
# Modify the connect method to use config with addr for specifying the port
public_url = ngrok.connect(addr="8501", bind_tls=True)
# bind_tls=True is necessary for streamlit which defaults to https
print(f"Public URL: {public_url}")

# Run the streamlit app
!cd series_analyzing_system && streamlit run streamlit_app.py

In [ ]:
!cd series_analyzing_system && python gradio_app.py

# Import Libraries

In [ ]:
import pandas as pd
import re
from datasets import Dataset

# Load the Dataset (Transcript) .csv

In [ ]:
data_path = 'series_analyzing_system/data/naruto.csv'
naruto_transcript_df = pd.read_csv(data_path)

In [ ]:
naruto_transcript_df.head()

,name,line
0,Naruto,(Laughing) Give it up. (Shows the stone faces...
1,Hiruzen,(Turns away from his writing) I hope you’re n...
2,Ninja,Naseer Sabah
3,Ninja,is the best person on earth
4,Naruto,muah


In [ ]:
# Remove the character in braket or action from transcript using re
def remove_paranthesis(text):
    result = re.sub(r'\(.*?\)', '', text)

    return result


In [ ]:
naruto_transcript_df['line'] = naruto_transcript_df['line'].apply(remove_paranthesis)

In [ ]:
naruto_transcript_df.head()

,name,line
0,Naruto,"Give it up. You’re just bent, because you d..."
1,Hiruzen,I hope you’re not bothering me with some tri...
2,Ninja,Naseer Sabah
3,Ninja,is the best person on earth
4,Naruto,muah


In [ ]:
# Get the response and make sure its not short
naruto_transcript_df["number_of_words"] = naruto_transcript_df["line"].str.strip().str.split("")
# Count it
naruto_transcript_df["number_of_words"] = naruto_transcript_df["number_of_words"].apply(lambda x: len(x))

In [ ]:
naruto_transcript_df.head()

,name,line,number_of_words
0,Naruto,"Give it up. You’re just bent, because you d...",131
1,Hiruzen,I hope you’re not bothering me with some tri...,86
2,Ninja,Naseer Sabah,14
3,Ninja,is the best person on earth,29
4,Naruto,muah,6


In [ ]:
# Naruto respond flag, so anything that has naruto and the word is greater than 5
naruto_transcript_df['naruto_respond_flag'] = 0
naruto_transcript_df.loc[(naruto_transcript_df['name'] == 'Naruto')&(naruto_transcript_df["number_of_words"]>5), "naruto_respond_flag"] = 1

In [ ]:
naruto_transcript_df

,name,line,number_of_words,naruto_respond_flag
0,Naruto,"Give it up. You’re just bent, because you d...",131,1
1,Hiruzen,I hope you’re not bothering me with some tri...,86,0
2,Ninja,Naseer Sabah,14,0
3,Ninja,is the best person on earth,29,0
4,Naruto,muah,6,1
...,...,...,...,...
158,Iruka,Congratulations. You graduate. Naruto’s stand...,177,0
159,Iruka,Huh?,6,0
160,Naruto,Iruka Sensei!,15,1
161,Iruka,Ah! That hurts!,17,0


In [ ]:
# Picking the indexes and Excluding the first row because its just intializing coversation
indexes_to_take = list(naruto_transcript_df[(naruto_transcript_df['naruto_respond_flag'] == 1)& (naruto_transcript_df.index>0)].index)

In [ ]:
indexes_to_take[:3]

[4, 6, 20]

In [ ]:
# Create the prompt we will feed through the chatbot so it can act as naruto, and give it the statement set to naruto and the response so it can imitate it

system_prompt = """" Your are naruto from the anime "Naruto". Your responses should reflect his personalities and speech patterns \n"""
prompts=[]
for ind in indexes_to_take:
    prompt = system_prompt

    # What was said
    prompt += naruto_transcript_df.iloc[ind-1]["line"] + "\n"
    # Response
    prompt += naruto_transcript_df.iloc[ind]["line"]
    prompts.append(prompt)

In [ ]:
print(prompts[5])

" Your are naruto from the anime "Naruto". Your responses should reflect his personalities and speech patterns 
  Cut the stupid tricks! This is your last warning!
 This sucks. Loser.


In [ ]:
# Puttint the prompts in pd df
df = pd.DataFrame({"prompt": prompts})
df.head()

,prompt
0,""" Your are naruto from the anime ""Naruto"". You..."
1,""" Your are naruto from the anime ""Naruto"". You..."
2,""" Your are naruto from the anime ""Naruto"". You..."
3,""" Your are naruto from the anime ""Naruto"". You..."
4,""" Your are naruto from the anime ""Naruto"". You..."


In [ ]:
# Convert to huggingface dataset
dataset = Dataset.from_pandas(df)